<a href="https://colab.research.google.com/github/severus-fan/Machine_Learning/blob/main/PM2_5_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Kaggle Task](https://www.kaggle.com/c/ml2020spring-hw1/submit)

In [ ]:
# OpenCV 套件，和 Google Drive 無關
import cv2
# import Google Drive 套件
from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('/content/gdrive/MyDrive/ML2020/hw1_regression/dataset/train.csv', encoding = 'big5')
data[data == 'NR'] = 0
data = data.iloc[:, 3:]

In [ ]:
raw_data = data.to_numpy()
raw_data

array([['14', '14', '14', ..., '15', '15', '15'],
       ['1.8', '1.8', '1.8', ..., '1.8', '1.8', '1.8'],
       ['0.51', '0.41', '0.39', ..., '0.35', '0.36', '0.32'],
       ...,
       ['36', '55', '72', ..., '118', '100', '105'],
       ['1.9', '2.4', '1.9', ..., '1.5', '2', '2'],
       ['0.7', '0.8', '1.8', ..., '1.6', '1.8', '2']], dtype=object)

In [ ]:
month_data = {}
for month in range(12):
  sample = np.empty([18,480])
  for day in range(20):
    sample[:,day*24:(day+1)*24] = raw_data[month*20*18+day*18:month*20*18+(day+1)*18,:]
  month_data[month] = sample

In [ ]:
x = np.empty([12*471,18*9], dtype=float)
y = np.empty([12*471,1], dtype=float)

for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 19 and hour > 14:
        continue
      x[month*471+day*24+hour,:] = month_data[month][:,day*24+hour:day*24+hour+9].reshape(1,-1)
      y[month*471+day*24+hour,0] = month_data[month][9,day*24+hour+9]
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [ ]:
mean_x = np.mean(x, axis = 0)
mean_x

array([2.25365180e+01, 2.25432413e+01, 2.25497877e+01, 2.25536801e+01,
       2.25559802e+01, 2.25562279e+01, 2.25562456e+01, 2.25552548e+01,
       2.25517162e+01, 1.70238854e+00, 1.70230007e+00, 1.70217622e+00,
       1.70214084e+00, 1.70201699e+00, 1.70196391e+00, 1.70189314e+00,
       1.70185775e+00, 1.70178698e+00, 3.89033970e-01, 3.88934890e-01,
       3.88959660e-01, 3.89166667e-01, 3.89334749e-01, 3.89370134e-01,
       3.89304671e-01, 3.89071125e-01, 3.88581033e-01, 1.40153928e-01,
       1.40187544e-01, 1.40438783e-01, 1.40796178e-01, 1.41013800e-01,
       1.41086341e-01, 1.41127035e-01, 1.41095188e-01, 1.40916490e-01,
       2.14743454e+00, 2.15033616e+00, 2.15251238e+00, 2.15507785e+00,
       2.15559094e+00, 2.15580326e+00, 2.15553786e+00, 2.15373319e+00,
       2.14856688e+00, 1.01164897e+01, 1.01188606e+01, 1.01283793e+01,
       1.01436837e+01, 1.01568117e+01, 1.01639597e+01, 1.01680290e+01,
       1.01668790e+01, 1.01600318e+01, 1.22496461e+01, 1.22547063e+01,
      

In [ ]:
std_x = np.std(x, axis = 0)
std_x

array([ 6.28492341,  6.28716061,  6.29020697,  6.29307145,  6.29572176,
        6.29854833,  6.30157151,  6.30461971,  6.30810055,  0.1258782 ,
        0.12585876,  0.12583988,  0.12584048,  0.12584956,  0.12587148,
        0.12588662,  0.12590121,  0.12590223,  0.32613624,  0.32609996,
        0.3260708 ,  0.32605536,  0.32606044,  0.32604933,  0.32604299,
        0.32600697,  0.32586854,  0.10445539,  0.1044395 ,  0.1046868 ,
        0.10506596,  0.10520688,  0.10519009,  0.1051768 ,  0.10514954,
        0.10508098,  2.29609706,  2.29791935,  2.2977232 ,  2.29828255,
        2.29814586,  2.29803601,  2.29774023,  2.29743508,  2.2929628 ,
        6.20168435,  6.20286429,  6.20735997,  6.2137359 ,  6.21724748,
        6.21522012,  6.21389434,  6.21248258,  6.21119177,  7.60534775,
        7.60802674,  7.61251565,  7.61762304,  7.61914967,  7.61659176,
        7.61521651,  7.61413145,  7.61097383, 18.77011328, 18.78673915,
       18.79668523, 18.79801691, 18.7956666 , 18.79071398, 18.78

In [ ]:
x.shape

(5652, 162)

In [ ]:
for i in range(len(x)):
  for j in range(len(x[0])):
    if std_x[j] != 0:
      x[i][j] = (x[i][j] - mean_x[j])/std_x[j]
x

array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

In [ ]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x_train_set = np.concatenate((np.ones([4521, 1]), x_train_set), axis = 1).astype(float)
learning_rate = 1
iter_time = 4000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x_train_set, w) - y_train_set, 2))/4521)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x_train_set.transpose(), np.dot(x_train_set, w) - y_train_set) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('/content/gdrive/MyDrive/ML2020/hw1_regression/weight.npy', w)

0:27.239591682144457
100:9.609281246675417
200:7.187052238424128
300:6.450125671920877
400:6.193309808253797
500:6.0771919745802085
600:6.008656278109068
700:5.960709639173118
800:5.924155848604971
900:5.89504347131374
1000:5.871258943430991
1100:5.851490106428933
1200:5.834848495978658
1300:5.820700412984782
1400:5.80857753341518
1500:5.798124135484113
1600:5.78906382782008
1700:5.781177671705409
1800:5.7742893456949265
1900:5.768254825800758
2000:5.7629550412897
2100:5.7582905367062915
2200:5.7541775148026835
2300:5.750544848622057
2400:5.747331786515197
2500:5.744486161506646
2600:5.741962974059872
2700:5.739723255801377
2800:5.7377331478949
2900:5.735963145756931
3000:5.73438747438755
3100:5.732983567514854
3200:5.731731630170514
3300:5.730614268995214
3400:5.729616178032223
3500:5.728723870356484
3600:5.727925447848262
3700:5.72721040292387
3800:5.726569447200932
3900:5.725994362987834


In [ ]:
x_validation = np.concatenate((np.ones([1131, 1]), x_validation), axis = 1).astype(float)
loss_val = np.sqrt(np.sum(np.power(np.dot(x_validation, w) - x_validation, 2))/1131)#rmse
print(str(loss))

5.725482770853422


### Results
| LR | IT | LOSS |
| - | - | - |
| 1 | 10000 | 5.719714393411634 |
| 1 | 4000 | 5.725482770853422 |
| 1 | 2000 | 5.763004742872873 |
| 1 | 1000 | 5.871475152255766 |
| 5 | 1000 | 5.884517862476503 |
| 10 | 1000 | 5.890358170199704 |
| 25 | 1000 | 5.897706913061451 |
| 50 | 1000 | 5.913572549094058 |
| 75 | 1000 | 5.938845073767831 |
| 100 | 1000 | 5.974024879930956 |
| 200 | 1000 | 6.211305992326504 |
| 200 | 1000 | 6.589653415894668 |


In [ ]:
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv("/content/gdrive/MyDrive/ML2020/hw1_regression/dataset/test.csv", header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

In [ ]:
w = np.load('/content/gdrive/MyDrive/ML2020/hw1_regression/weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[  6.76204789],
       [ 18.0556907 ],
       [ 23.96695258],
       [  7.19642642],
       [ 27.29311389],
       [ 22.47165644],
       [ 24.21879679],
       [ 30.89043679],
       [ 16.83297905],
       [ 59.96382501],
       [ 12.07951722],
       [  9.21779575],
       [ 62.64119763],
       [ 53.6052402 ],
       [ 21.95547559],
       [ 12.13882541],
       [ 32.25029231],
       [ 66.41780131],
       [ -1.07164071],
       [ 17.06790646],
       [ 41.80476973],
       [ 72.05915185],
       [  8.93454106],
       [ 17.70290852],
       [ 14.81895021],
       [ 38.21400541],
       [ 14.22988962],
       [ 72.65454235],
       [  6.95174621],
       [ 55.829812  ],
       [ 24.19581542],
       [  9.33716592],
       [  3.24350562],
       [ 19.13610125],
       [ 29.22247704],
       [ 36.95234399],
       [ 43.22868173],
       [ 30.58867444],
       [ 41.70220128],
       [ 35.48215663],
       [  7.03407913],
       [ 41.19237195],
       [ 31.28905018],
       [ 51

In [ ]:
import csv
with open('/content/gdrive/MyDrive/ML2020/hw1_regression/submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 6.762047892509877]
['id_1', 18.05569069535842]
['id_2', 23.96695257581583]
['id_3', 7.196426416743855]
['id_4', 27.293113890722953]
['id_5', 22.471656436818233]
['id_6', 24.218796785338238]
['id_7', 30.89043679209179]
['id_8', 16.8329790480385]
['id_9', 59.963825010504]
['id_10', 12.079517223971749]
['id_11', 9.217795748344408]
['id_12', 62.641197630216276]
['id_13', 53.605240202760406]
['id_14', 21.955475586356716]
['id_15', 12.138825414716495]
['id_16', 32.250292306074016]
['id_17', 66.41780131288769]
['id_18', -1.071640712267636]
['id_19', 17.067906459128125]
['id_20', 41.80476972765859]
['id_21', 72.05915185100343]
['id_22', 8.934541057039231]
['id_23', 17.70290852401028]
['id_24', 14.818950208682619]
['id_25', 38.21400540617245]
['id_26', 14.229889616525055]
['id_27', 72.65454235084557]
['id_28', 6.951746211857175]
['id_29', 55.82981199768028]
['id_30', 24.19581541854001]
['id_31', 9.337165923469838]
['id_32', 3.2435056187131104]
['id_33', 19.1361012459993